# Indian Regional Recommendation Systems

The goal for a recommendation system is to extract information from data about the relationship existing between users and products. One of the common usage is to take the products that the user already likes and and answer the question "What other products can be recommended to the user?"

There are multiple ways of answering the above question. These are the three main types algorithms being used for recommendation systems, to try answering different aspects of recommendations:

- **Content Based Filtering**
  - Used to find products with "similar" attributes. Example: if a person likes movies from action genere then recommend other books from action genere.
  - The movie will have different attributes, such as, direction, cast, cinematography, story etc. A field specialist will rate all these attributes on a fixed scale.
  - Based on similarity between these attributes, we can recommend movies to a new user once they have picked up a few movies that they liked.
  - One major drawback of this method is getting a field expert to rate movies on their attributes. It is time conuming and inefficient.
  
- **Collaberative Filtering**
  - This is somewhat indirect way of recommending, but is the most used method in the industry. Here we find products liked by "similar" users (they have the same interest as the active user) and recommend them to the active user. 
  - For an active user, who has rated some movies highly in the system; we find other users who have rated these movies in a similar manner (they might have rated other movies as well). These users have commen interests, so we can use other user's ratings as a guide for the likeliness of movie recommendation to the active user.
  - Here the assumption is that the movies that "similar" users like are similar to each other. In this method also we are measuring similarity of products, however here the similarity is measured indirectly through similarity of users.
- **Association Rules Learning**
  - Here we recommend "complimentary" products, for example, if someone is buying a smartphone then recommend a back cover or a tempered glass etc.
  - In movie recommendation, the products are substitutable to each other. However, in the above example of smartphone, the products are complimentary.
  - This algorithm tries to answer the question, "If a person likes to buy smartphones, along with it, what else would they like to buy?" i.e "Which products are *associated* with each other?"

### Get movies data from file into DataFrame

In [1]:
# Import some basic libraries in python for data preprocessing

import json
import numpy as np
import pandas as pd

In [3]:
# Read movies.json file and load into moviesData

moviesData = []

with open('archive/movies.json') as moviesFile:
    for line in moviesFile:
        moviesData.append(json.loads(line))

# Now take the moviesData and create a DataFrame

moviesDF = pd.DataFrame.from_records(moviesData)

# View the head of the moviesDF DataFrame

moviesDF.head()

,_id,rating,movie_id,description,language,released,rating_cnt,writer,director,cast,img_src,genre,inserted,name
0,{'$oid': '581994c40ad09b0701a0e727'},7.9,tt5286444,Neerja is the story of the courageous Neerja B...,[Hindi],{'$date': '2016-02-19T05:30:00.000+0530'},0,"[Saiwyn Quadras, Sanyukta Shaikh Chawla]",[Ram Madhvani],"[Sonam Kapoor, Shabana Azmi, Yogendra Tikku, A...",https://images-na.ssl-images-amazon.com/images...,"[Biography, Drama, Thriller]",{'$date': '2016-11-02T18:24:52.889+0530'},Neerja
1,{'$oid': '581994c40ad09b0701a0e728'},7.9,tt4434004,A story that revolves around drug abuse in the...,"[Hindi, Panjabi]",{'$date': '2016-06-17T05:30:00.000+0530'},0,"[Sudip Sharma, Abhishek Chaubey]",[Abhishek Chaubey],"[Alia Bhatt, Shahid Kapoor, Diljit Dosanjh, Ka...",https://images-na.ssl-images-amazon.com/images...,"[Crime, Drama, Thriller]",{'$date': '2016-11-02T18:24:52.938+0530'},Udta Punjab
2,{'$oid': '581994c50ad09b0701a0e729'},7.5,tt0248126,Yashvardhan Raichand lives a very wealthy life...,"[Hindi, English, Urdu]",{'$date': '2001-12-14T05:30:00.000+0530'},0,"[Karan Johar, Sheena Parikh]",[Karan Johar],"[Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Kha...",https://images-na.ssl-images-amazon.com/images...,"[Drama, Musical, Romance]",{'$date': '2016-11-02T18:24:53.046+0530'},Kabhi Khushi Kabhie Gham...
3,{'$oid': '581994c50ad09b0701a0e72a'},8.0,tt0347304,"Naina, an introverted, perpetually depressed g...","[Hindi, Urdu, Gujarati, Panjabi]",{'$date': '2003-11-28T05:30:00.000+0530'},0,"[Niranjan Iyengar, Karan Johar]",[Nikkhil Advani],"[Shah Rukh Khan, Preity Zinta, Saif Ali Khan, ...",https://images-na.ssl-images-amazon.com/images...,"[Comedy, Drama, Romance]",{'$date': '2016-11-02T18:24:53.052+0530'},Kal Ho Naa Ho
4,{'$oid': '581994c50ad09b0701a0e72b'},7.6,tt3043252,'Parched' is a story about women set in the he...,"[Hindi, English]",{'$date': '2016-06-17T05:30:00.000+0530'},0,"[Supratik Sen, Leena Yadav]",[Leena Yadav],"[Tannishtha Chatterjee, Radhika Apte, Lehar Kh...",https://images-na.ssl-images-amazon.com/images...,[Drama],{'$date': '2016-11-02T18:24:53.572+0530'},Parched


## Building a content based recommender system

Here we will attempt to built two types of content based recommendation systems

* **Plot Based Recommendation System**: This model takes movie descriptions and taglines into consideration and provides recommendation with similar plot descriptions.

* **Metadata Based Recommendation System**: This model takes different features such as, genres, keywords, cast, and crew etc into consideration and provides recommendations that are most similar.

## I. Plot Based Recommendation System

**Goal:** Compute the similarity matrix of all the movies with each other, based on their plot text (using pairwise cosine similarity method)

**Approach:** Represent plot text (or documents) as vectors i.e a series of numbers, where each number/dimension represents occurance of a specific word in the vocabulary. The size of the vocabulary vector is the number of unique words present when all the documents are put together.

**Methods:** To create these vectors for all the movies, we use vectorizers;
  - Count Vectorizer
  - TF-IDF Vectorizer

#### Count Vectorizer

This is the simplest type of vectorizer. 

Consider an example, where we have three documents (texts)

A: The sun is a star

B: My love is like a red, red rose

C: Mary had a little lamb

Now our goal is to convert these documents into vectors.

**Step-1**

We first compute the vocabulary; i.e the vector comprising of all the unique words present across all the documents.

V = (the, sun, is, a, star, my, love, like, red, rose, mary, had, little, lamb)

The size of this vocabulary is 14.

__Special__

A common practice is to remove common words e.g. a, the, is, had, my etc.. These are also called *'Stop Words'*.

After removing these stop words;

V: (like, little, lamb, love, mary, red, rose, sun, star)

**Step-2**

Now each document is interpreted as a vector of size 9, where each dimension represents the number of times each word occurs.

So, using the CountVectorizer approach, A, B and C will be represented as:

A: (0, 0, 0, 0, 0, 0, 0, 1, 1)

B: (1, 0, 0, 1, 0, 2, 1, 0, 0)

C: (0, 1, 1, 0, 1, 0, 0, 0, 0)

#### TF-IDF  Vectorizer

Not all words in a document carry equal weight, (for example we saw that stop words have no weightage at all).

**TF-IDF: Term Frequency - Inverse Document Frequency**

It assigns the weights to each word according to the formula (for every word i in document j):

$$ w_{i,j} = tf_{i,j} \times log\bigg(\frac{N}{df_i} \bigg) $$

Where,

  $w_{i,j}$ is the weight of word i in document j

  $tf_{i,j}$ is the term frequency for word i in document j

  $N$ is the total number of documents

  $df_i$ is the number of documents that contain the term i
  
<br>

**pro**: Speeds up the computation of cossine similarity score

### Cosine Similarity:

Cosine similarity between two documents, x and y,

$$ cosine(x, y) = \frac{x.y^T}{||x||.||y||} $$

It takes a value between -1 and 1, higher the score, more similar the documents are to each other.

### Steps for building a Plot based Recommender

It takes a movie title as an argument and outputs a list of movies that are most similar based on their plots.

**Steps**

1. Clean the data to the format required to build the model
2. Create TF-IDF vectors for the plot description of every movie
3. Compute the pairwise cosine similarity between every movie
4. Write a recommender function that takes in a movie title as an argument and outputs movies most similar to it based on the plot.

### 1. Prep the data

In [4]:
# Let's create a dataframe with name and description
plotDF = moviesDF.loc[:, ['name', 'description']]

# Convert both columns to lowercase
plotDF.loc[:, 'name'] = plotDF.loc[:, 'name'].apply(lambda x: x.lower())
plotDF.loc[:, 'description'] = plotDF.loc[:, 'description'].apply(lambda x: x.lower())

# Drop all the rows which are empty
plotDF = plotDF[plotDF['description'] != '']

# Now drop duplicates in the plotDF
plotDF = plotDF.drop_duplicates()

plotDF.head()

,name,description
0,neerja,neerja is the story of the courageous neerja b...
1,udta punjab,a story that revolves around drug abuse in the...
2,kabhi khushi kabhie gham...,yashvardhan raichand lives a very wealthy life...
3,kal ho naa ho,"naina, an introverted, perpetually depressed g..."
4,parched,'parched' is a story about women set in the he...


In [5]:
# Lets quickly view the info of plotDF
plotDF.info()

# There are 2850 unique movies, we can see that there are no null description

<class 'pandas.core.frame.DataFrame'>
Index: 1751 entries, 0 to 2847
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         1751 non-null   object
 1   description  1751 non-null   object
dtypes: object(2)
memory usage: 41.0+ KB


### 2. Create the TF-IDF matrix

Each row of this matrix represents the TF_IDF vector of the description feature of the corresponding movie in the plotDF data frame.

In [6]:
# Import the TfIdfVectorizer from scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF IDF Vectorizer Object
# with the removal of english stopwords turned on
tfidf = TfidfVectorizer(stop_words = 'english')

# Now costruct the TF-IDF Matrix by applying the fit_transform method on the description feature
tfidf_matrix = tfidf.fit_transform(plotDF['description'])

# View the shape of the TfIdf_matrix
tfidf_matrix.shape

(1751, 7702)

### 3. Computing the cosine similarity score

Here we are going to create a matrix of size 1751 x 1751, where i-th row and j-th column column represents the similarity score between movies i and j.

This matrix will be symmetric in nature and all the elements along the diagonal will be 1, since it the similarity score of the matrix with itself.

Also, as we have represented the movie plots as TF-IDF vectors, they have the magnitude of 1. So, we need not calculate the magnitude of the dot product (denominator of the cosine similarity function will always be 1).

So out cosine similarity function reduces to:

$$cosine(x, y) = x.y^T$$



In [7]:
# Import linear_kernel from scikit-learn to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix by using linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Let's view the shape of the cosine similarity matrix
print("Shape of cosine similarity matrix: ", cosine_sim.shape)

# Let's quickly view how the matrix looks like in first few column and rows
cosine_sim[0:5, 0:5]

Shape of cosine similarity matrix:  (1751, 1751)


array([[1.        , 0.00849819, 0.01322405, 0.02102451, 0.00729834],
       [0.00849819, 1.        , 0.        , 0.        , 0.00940967],
       [0.01322405, 0.        , 1.        , 0.01381885, 0.        ],
       [0.02102451, 0.        , 0.01381885, 1.        , 0.        ],
       [0.00729834, 0.00940967, 0.        , 0.        , 1.        ]])

In [8]:
# Construct a pandas series of movie name as index and index as value
indices = pd.Series(plotDF.index, index = plotDF['name'])
indices.head()

name
neerja                         0
udta punjab                    1
kabhi khushi kabhie gham...    2
kal ho naa ho                  3
parched                        4
dtype: int64

#### 4. Build the recommender function

**Steps**

1. Input the title of the movie as an argument
2. Obtain the index of the movie from the *indeces* series
3. Get the list of cosine similarity scores for that particular movie with all movies using cosine_sim matrix.And, convert this in to a list of tuples where the first element is the position and second element is the similarity score.
4. Sort this list of tuples on the basis of the cosine similarity scores.
5. Get the top 10 elements of the list, while ignoring the first element as it refers to the similarity score with itself.
6. Return the titles corresponding to the indices of the top 10 elements.

In [9]:
# Function for returning top 10 movies for a movie title as input
def plot_based_recommender(title, df = plotDF, cosine_sim = cosine_sim, indices = indices):
  # Convert title to lower-case
  title = title.lower()

  # Obtain the index of the movie that matched the title 
  try:
    idx = indices[title]
  except KeyError:
    print('Movie does not exist :(')
    return False

  # Get the pairwise similarity score of all the movies with that movie
  # and convert it into a list of tuples (position, similarity score)
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Sort the movies based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  # Get the scores of the top 10 most similar movies. Ignore the first movie.
  sim_scores = sim_scores[1:11]

  # get the movie indices
  movie_indices = [sim_score[0] for sim_score in sim_scores]

  # Return the top 10 similar movies
  return df['name'].iloc[movie_indices]

In [10]:
plot_based_recommender('Pyaar Ka Punchnama')

105      pyaar ka punchnama 2
72                delhi belly
2051         mahakavi kalidas
2126                   crayon
164        in the name of god
3               kal ho naa ho
503             apradhi kaun?
360      chalti ka naam gaadi
1760    shubhamangal savadhan
2019               orr eravuu
Name: name, dtype: object

## II. Metadata Based Recommender System

To build this model, we will be using the following meta-data:

- genre
- language
- director
- cast
- smovie description

Apart from the difference in data being used, we will largely follow the same steps as the plot based recommender system.

### Prep the Data

In [11]:
# Prepare the data
metaDF = moviesDF[['name', 'genre', 'language', 'director', 'cast', 'description']]
metaDF.head()

,name,genre,language,director,cast,description
0,Neerja,"[Biography, Drama, Thriller]",[Hindi],[Ram Madhvani],"[Sonam Kapoor, Shabana Azmi, Yogendra Tikku, A...",Neerja is the story of the courageous Neerja B...
1,Udta Punjab,"[Crime, Drama, Thriller]","[Hindi, Panjabi]",[Abhishek Chaubey],"[Alia Bhatt, Shahid Kapoor, Diljit Dosanjh, Ka...",A story that revolves around drug abuse in the...
2,Kabhi Khushi Kabhie Gham...,"[Drama, Musical, Romance]","[Hindi, English, Urdu]",[Karan Johar],"[Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Kha...",Yashvardhan Raichand lives a very wealthy life...
3,Kal Ho Naa Ho,"[Comedy, Drama, Romance]","[Hindi, Urdu, Gujarati, Panjabi]",[Nikkhil Advani],"[Shah Rukh Khan, Preity Zinta, Saif Ali Khan, ...","Naina, an introverted, perpetually depressed g..."
4,Parched,[Drama],"[Hindi, English]",[Leena Yadav],"[Tannishtha Chatterjee, Radhika Apte, Lehar Kh...",'Parched' is a story about women set in the he...


In [12]:
# We want to keep only the first 3 genre and cast (actors) in the list format
# We want to keep only the first director

metaDF.loc[:, 'genre'] = metaDF.loc[:, 'genre'].apply(lambda x: x[:3] if len(x) > 3 else x)
metaDF.loc[:, 'cast'] = metaDF.loc[:, 'cast'].apply(lambda x: x[:3] if len(x) > 3 else x)
metaDF.loc[:, 'director'] = metaDF.loc[:, 'director'].apply(lambda x: x[:1])
metaDF.head()

,name,genre,language,director,cast,description
0,Neerja,"[Biography, Drama, Thriller]",[Hindi],[Ram Madhvani],"[Sonam Kapoor, Shabana Azmi, Yogendra Tikku]",Neerja is the story of the courageous Neerja B...
1,Udta Punjab,"[Crime, Drama, Thriller]","[Hindi, Panjabi]",[Abhishek Chaubey],"[Alia Bhatt, Shahid Kapoor, Diljit Dosanjh]",A story that revolves around drug abuse in the...
2,Kabhi Khushi Kabhie Gham...,"[Drama, Musical, Romance]","[Hindi, English, Urdu]",[Karan Johar],"[Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Khan]",Yashvardhan Raichand lives a very wealthy life...
3,Kal Ho Naa Ho,"[Comedy, Drama, Romance]","[Hindi, Urdu, Gujarati, Panjabi]",[Nikkhil Advani],"[Shah Rukh Khan, Preity Zinta, Saif Ali Khan]","Naina, an introverted, perpetually depressed g..."
4,Parched,[Drama],"[Hindi, English]",[Leena Yadav],"[Tannishtha Chatterjee, Radhika Apte, Lehar Khan]",'Parched' is a story about women set in the he...


The spaces between the names of actors and directors can create problems as they can be considered as separate words. We do not want that, so our next step is to strip the spaces between the names.

In [13]:
def sanitize(x):
    if isinstance(x, list):
        # Strip spaces
        return [i.replace(" ", "") for i in x]
    else:
        # if it is empty, return an empty string
        if isinstance(x, str):
            return x.replace(" ", "")
        else: 
            return ''

In [14]:
#Apply the generate_list function to cast, keywords, director and genres 
for feature in ['director', 'cast']:
    metaDF[feature] = metaDF[feature].apply(sanitize)

/var/folders/ww/h3r9s2j93ylbk0pdctkmhyzm0000gq/T/ipykernel_97165/2226031449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metaDF[feature] = metaDF[feature].apply(sanitize)


In [15]:
metaDF.head()

,name,genre,language,director,cast,description
0,Neerja,"[Biography, Drama, Thriller]",[Hindi],[RamMadhvani],"[SonamKapoor, ShabanaAzmi, YogendraTikku]",Neerja is the story of the courageous Neerja B...
1,Udta Punjab,"[Crime, Drama, Thriller]","[Hindi, Panjabi]",[AbhishekChaubey],"[AliaBhatt, ShahidKapoor, DiljitDosanjh]",A story that revolves around drug abuse in the...
2,Kabhi Khushi Kabhie Gham...,"[Drama, Musical, Romance]","[Hindi, English, Urdu]",[KaranJohar],"[AmitabhBachchan, JayaBhaduri, ShahRukhKhan]",Yashvardhan Raichand lives a very wealthy life...
3,Kal Ho Naa Ho,"[Comedy, Drama, Romance]","[Hindi, Urdu, Gujarati, Panjabi]",[NikkhilAdvani],"[ShahRukhKhan, PreityZinta, SaifAliKhan]","Naina, an introverted, perpetually depressed g..."
4,Parched,[Drama],"[Hindi, English]",[LeenaYadav],"[TannishthaChatterjee, RadhikaApte, LeharKhan]",'Parched' is a story about women set in the he...


In [16]:
# Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['genre']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['cast'] + [x['name']] + [x['description']])

# Create the new soup feature 
metaDF['soup'] = metaDF.apply(create_soup, axis=1)

#Display the soup of the first movie 
metaDF.iloc[0]['soup']

/var/folders/ww/h3r9s2j93ylbk0pdctkmhyzm0000gq/T/ipykernel_97165/199222214.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metaDF['soup'] = metaDF.apply(create_soup, axis=1)


'Biography Drama Thriller RamMadhvani SonamKapoor ShabanaAzmi YogendraTikku Neerja Neerja is the story of the courageous Neerja Bhanot, who sacrificed her life while protecting the lives of 359 passengers on the Pan Am flight 73 in 1986. The flight was hijacked by a terrorist organization.'

In [17]:
# Let's create a dataframe with name and soup
soupDF = metaDF.loc[:, ['name', 'soup']]

# Convert both columns to lowercase
soupDF.loc[:, 'name'] = soupDF.loc[:, 'name'].apply(lambda x: x.lower())
soupDF.loc[:, 'soup'] = soupDF.loc[:, 'soup'].apply(lambda x: x.lower())

# Drop all the rows which are empty
soupDF = soupDF[soupDF['soup'] != '']

# Now drop duplicates in the soupDF
soupDF = soupDF.drop_duplicates()

soupDF.head()
print(soupDF.shape)

(2850, 2)


In [18]:
# Import the TfIdfVectorizer from scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF IDF Vectorizer Object
# with the removal of english stopwords turned on
tfidf = TfidfVectorizer(stop_words = 'english')

# Now costruct the TF-IDF Matrix by applying the fit_transform method on the description feature
tfidf_matrix = tfidf.fit_transform(soupDF['soup'])

# View the shape of the TfIdf_matrix
tfidf_matrix.shape

(2850, 15373)

In [19]:
# Import linear_kernel from scikit-learn to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix by using linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Let's view the shape of the cosine similarity matrix
print("Shape of cosine similarity matrix: ", cosine_sim.shape)

# Let's quickly view how the matrix looks like in first few column and rows
cosine_sim[0:5, 0:5]

Shape of cosine similarity matrix:  (2850, 2850)


array([[1.        , 0.01715787, 0.01183263, 0.01769827, 0.00725645],
       [0.01715787, 1.        , 0.00183587, 0.00165802, 0.00912824],
       [0.01183263, 0.00183587, 1.        , 0.04098887, 0.00158062],
       [0.01769827, 0.00165802, 0.04098887, 1.        , 0.0014275 ],
       [0.00725645, 0.00912824, 0.00158062, 0.0014275 , 1.        ]])

In [20]:
# Construct a pandas series of movie name as index and index as value
indices = pd.Series(soupDF.index, index = soupDF['name'])
indices.head()

name
neerja                         0
udta punjab                    1
kabhi khushi kabhie gham...    2
kal ho naa ho                  3
parched                        4
dtype: int64

In [21]:
# Function for returning top 10 movies for a movie title as input
def plot_based_recommender(title, df = soupDF, cosine_sim = cosine_sim, indices = indices):
  # Convert title to lower-case
  title = title.lower()

  # Obtain the index of the movie that matched the title 
  try:
    idx = indices[title]
  except KeyError:
    print('Movie does not exist :(')
    return False

  # Get the pairwise similarity score of all the movies with that movie
  # and convert it into a list of tuples (position, similarity score)
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Sort the movies based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  # Get the scores of the top 10 most similar movies. Ignore the first movie.
  sim_scores = sim_scores[1:11]

  # get the movie indices
  movie_indices = [sim_score[0] for sim_score in sim_scores]

  # Return the top 10 similar movies
  return df['name'].iloc[movie_indices]

In [25]:
plot_based_recommender('Swades')

2813                      raktha kanneeru
2526                             yeh aman
1729                     carry on maratha
356                              haqeeqat
391                                junoon
383                                  holi
8                         my name is khan
2558                   adi shankaracharya
7       lagaan: once upon a time in india
45                             veer-zaara
Name: name, dtype: object

In [23]:
# database 
db = {} 
db['cosine_sim'] = cosine_sim
db['indices'] = indices

import pickle

dbfile = open('examplePickle', 'ab') 
pickle.dump(db, dbfile)                      
dbfile.close() 